In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

PROJECT_ROOT = Path("..")
PROC_DIR = PROJECT_ROOT / "data" / "processed"
PROC_DIR


WindowsPath('../data/processed')

In [ ]:
#------------------------------------------------------------------------------------------------------------------------
#Loading the cleaned data for each ETF. We look for parquet first, then csv, and raise an error if neither is found.
# This allows us to easily switch between formats and ensures we don't accidentally load old, uncleaned data.
#------------------------------------------------------------------------------------------------------------------------

def load_clean(etf: str) -> pd.DataFrame:
    pqt = PROC_DIR / f"{etf.lower()}_clean.parquet"
    csv = PROC_DIR / f"{etf.lower()}_clean.csv"
    if pqt.exists():
        return pd.read_parquet(pqt)
    if csv.exists():
        return pd.read_csv(csv)
    raise FileNotFoundError(f"Missing clean file for {etf}. Looked for {pqt.name} and {csv.name}")

spy  = load_clean("SPY")
voo  = load_clean("VOO")
qqq  = load_clean("QQQ")
schd = load_clean("SCHD")

for name, df in [("SPY", spy), ("VOO", voo), ("QQQ", qqq), ("SCHD", schd)]:
    print(name, df.shape, "weight sum:", float(df["weight"].sum()))


SPY (504, 4) weight sum: 0.99965068
VOO (498, 4) weight sum: 0.9973
QQQ (103, 4) weight sum: 0.9996999999999999
SCHD (99, 4) weight sum: 0.9990999999999999


In [ ]:
#---------------------------------------------------------------------------------------------------------
# Combine all holdings into a single DataFrame for analysis. We can always filter down to specific ETFs later.
#---------------------------------------------------------------------------------------------------------

holdings = pd.concat([spy, voo, qqq, schd], ignore_index=True)

# Standardize holding_name again just to be safe
holdings["holding_name"] = holdings["holding_name"].astype(str).str.strip().str.upper()
holdings["etf"] = holdings["etf"].astype(str).str.strip().str.upper()

# Sanity
holdings.head(), holdings["etf"].value_counts()


(   etf       holding_name ticker    weight
 0  SPY        NVIDIA CORP   NVDA  0.077702
 1  SPY          APPLE INC   AAPL  0.066152
 2  SPY     MICROSOFT CORP   MSFT  0.051086
 3  SPY     AMAZON.COM INC   AMZN  0.033212
 4  SPY  ALPHABET INC CL A  GOOGL  0.030750,
 etf
 SPY     504
 VOO     498
 QQQ     103
 SCHD     99
 Name: count, dtype: int64)

In [ ]:
# Create a pivot table with holding_name as index, etf as columns, and weight as values. Missing values are filled with 0.

W = holdings.pivot_table(
    index="holding_name",
    columns="etf",
    values="weight",
    aggfunc="sum",
    fill_value=0.0
)

W.shape, W.head()


((737, 4),
 etf                  QQQ    SCHD       SPY     VOO
 holding_name                                      
 1ST SOURCE CORP      0.0  0.0005  0.000000  0.0000
 3M CO                0.0  0.0000  0.001581  0.0014
 A O SMITH CORP       0.0  0.0000  0.000000  0.0001
 ABBOTT LABORATORIES  0.0  0.0000  0.003315  0.0032
 ABBVIE INC           0.0  0.0350  0.006876  0.0066)

In [ ]:
# Create a pivot table with holding_name as index, etf as columns, and weight as values. Missing values are filled with 0.

W = holdings.pivot_table(
    index="holding_name",
    columns="etf",
    values="weight",
    aggfunc="sum",
    fill_value=0.0
)

W.shape, W.head()


((737, 4),
 etf                  QQQ    SCHD       SPY     VOO
 holding_name                                      
 1ST SOURCE CORP      0.0  0.0005  0.000000  0.0000
 3M CO                0.0  0.0000  0.001581  0.0014
 A O SMITH CORP       0.0  0.0000  0.000000  0.0001
 ABBOTT LABORATORIES  0.0  0.0000  0.003315  0.0032
 ABBVIE INC           0.0  0.0350  0.006876  0.0066)

In [ ]:
# Now we have a matrix W where rows are holdings, columns are ETFs, and values are weights. 
# We can analyze overlaps by looking at which rows have nonzero values in multiple columns.

etfs = list(W.columns)

present = (W > 0).astype(int)

overlap_count = present.T @ present
overlap_count


etf,QQQ,SCHD,SPY,VOO
etf,,,,
QQQ,103,4,60,85
SCHD,4,99,30,27
SPY,60,30,504,349
VOO,85,27,349,496


In [ ]:
# To get the percentage overlap, we can divide the overlap_count by the number of holdings in each ETF.

holding_counts = present.sum(axis=0)  # per ETF
overlap_pct = overlap_count.copy().astype(float)

for a in etfs:
    overlap_pct.loc[a, :] = overlap_pct.loc[a, :] / holding_counts[a]

(overlap_pct * 100).round(1)


etf,QQQ,SCHD,SPY,VOO
etf,,,,
QQQ,100.0,3.9,58.3,82.5
SCHD,4.0,100.0,30.3,27.3
SPY,11.9,6.0,100.0,69.2
VOO,17.1,5.4,70.4,100.0


In [ ]:
# The above gives us the percentage of holdings in ETF A that are also in ETF B, ignoring weights.
# To incorporate weights, we can calculate a weighted overlap by summing the minimum weight of each holding across the two ETFs.
# For example, if a holding has a weight of 2% in ETF A and 3% in ETF B, it contributes 2% to the weighted overlap (the minimum of the two weights).


weighted_overlap = pd.DataFrame(index=etfs, columns=etfs, dtype=float)

for a in etfs:
    for b in etfs:
        weighted_overlap.loc[a, b] = np.minimum(W[a], W[b]).sum()

weighted_overlap.round(4)


,QQQ,SCHD,SPY,VOO
QQQ,0.9997,0.0416,0.3931,0.4630
SCHD,0.0416,0.9991,0.0519,0.0464
SPY,0.3931,0.0519,0.9997,0.6523
VOO,0.4630,0.0464,0.6523,0.9973


In [ ]:
# To get the weighted percentage overlap, we can divide the weighted_overlap by the total weight of each ETF  
# (which should be 1.0 if weights are normalized).

total_weights = W.sum(axis=0)  # per ETF

weighted_overlap_pct = weighted_overlap.copy().astype(float)

for a in etfs:
    weighted_overlap_pct.loc[a, :] = weighted_overlap_pct.loc[a, :] / total_weights[a]

(weighted_overlap_pct * 100).round(2)


,QQQ,SCHD,SPY,VOO
QQQ,99.97,4.16,39.31,46.30
SCHD,4.16,99.91,5.19,4.64
SPY,39.31,5.19,99.97,65.23
VOO,46.30,4.64,65.23,99.73


In [ ]:
#Top overlap function to show the top N holdings contributing to the overlap between two ETFs, sorted by their contribution.

def top_overlap(a: str, b: str, n: int = 15) -> pd.DataFrame:
    df = pd.DataFrame({
        "holding_name": W.index,
        f"w_{a}": W[a].values,
        f"w_{b}": W[b].values,
    })
    df["overlap_contrib"] = np.minimum(df[f"w_{a}"], df[f"w_{b}"])
    df = df[df["overlap_contrib"] > 0].sort_values("overlap_contrib", ascending=False)
    return df.head(n).reset_index(drop=True)

top_overlap("SPY", "VOO", 15)


,holding_name,w_SPY,w_VOO,overlap_contrib
0,NVIDIA CORP,0.077702,0.0783,0.077702
1,APPLE INC,0.066152,0.0646,0.064600
2,MICROSOFT CORP,0.051086,0.0539,0.051086
3,AMAZON.COM INC,0.033212,0.0392,0.033212
4,BROADCOM INC,0.026750,0.0264,0.026400
5,TESLA INC,0.020045,0.0204,0.020045
6,EXXON MOBIL CORP,0.010817,0.0100,0.010000
7,WALMART INC,0.010025,0.0088,0.008800
8,MICRON TECHNOLOGY INC,0.007949,0.0078,0.007800
9,COSTCO WHOLESALE CORP,0.007572,0.0070,0.007000


In [ ]:
#top_overlap function to show the top N holdings contributing to the overlap between two ETFs, sorted by their contribution.
top_overlap("SPY", "QQQ", 15)


,holding_name,w_SPY,w_QQQ,overlap_contrib
0,NVIDIA CORP,0.077702,0.0875,0.077702
1,APPLE INC,0.066152,0.0744,0.066152
2,MICROSOFT CORP,0.051086,0.0587,0.051086
3,AMAZON.COM INC,0.033212,0.0418,0.033212
4,BROADCOM INC,0.026750,0.0302,0.026750
5,TESLA INC,0.020045,0.0410,0.020045
6,WALMART INC,0.010025,0.0356,0.010025
7,MICRON TECHNOLOGY INC,0.007949,0.0252,0.007949
8,COSTCO WHOLESALE CORP,0.007572,0.0245,0.007572
9,NETFLIX INC,0.005498,0.0177,0.005498


In [ ]:
#top_overlap function to show the top N holdings contributing to the overlap between two ETFs, sorted by their contribution.

top_overlap("QQQ", "SCHD", 15)


,holding_name,w_QQQ,w_SCHD,overlap_contrib
0,CISCO SYSTEMS INC,0.0165,0.0342,0.0165
1,PEPSICO INC,0.0123,0.0400,0.0123
2,AMGEN INC,0.0109,0.0376,0.0109
3,PAYCHEX INC,0.0019,0.0109,0.0019


In [ ]:
#portfolio_weights is a dictionary that defines the weight of each ETF in the portfolio. 
#In this example, we have an equal-weighted portfolio with 25% in each of the four ETFs: SPY, VOO, QQQ, and SCHD.

portfolio_weights = {
    "SPY": 0.25,
    "VOO": 0.25,
    "QQQ": 0.25,
    "SCHD": 0.25,
}

# Portfolio holding weights = sum ETF_weight * holding_weight
port = pd.Series(0.0, index=W.index)
for etf, w in portfolio_weights.items():
    port += w * W[etf]

port = port.sort_values(ascending=False)

port.head(20).to_frame("portfolio_weight").assign(pct=lambda x: (x["portfolio_weight"]*100).round(2))


,portfolio_weight,pct
holding_name,,
NVIDIA CORP,0.060876,6.09
APPLE INC,0.051288,5.13
MICROSOFT CORP,0.040921,4.09
AMAZON.COM INC,0.028553,2.86
BROADCOM INC,0.020837,2.08
TESLA INC,0.020361,2.04
META PLATFORMS INC,0.015825,1.58
CISCO SYSTEMS INC,0.015239,1.52
PEPSICO INC,0.014927,1.49


In [ ]:
#quick overlap summary table (clean for write-up)

summary = (weighted_overlap * 100).round(2)
summary


,QQQ,SCHD,SPY,VOO
QQQ,99.97,4.16,39.31,46.30
SCHD,4.16,99.91,5.19,4.64
SPY,39.31,5.19,99.97,65.23
VOO,46.30,4.64,65.23,99.73


In [ ]:
#how redundant is my portfolio? What percentage of the total weight is concentrated in the top 10, 20, or 50 holdings?

top10_share = port.head(10).sum()
top20_share = port.head(20).sum()
top50_share = port.head(50).sum()

{
    "Top 10 holdings share (%)": round(top10_share*100, 2),
    "Top 20 holdings share (%)": round(top20_share*100, 2),
    "Top 50 holdings share (%)": round(top50_share*100, 2),
}


{'Top 10 holdings share (%)': np.float64(28.37),
 'Top 20 holdings share (%)': np.float64(40.78),
 'Top 50 holdings share (%)': np.float64(60.61)}